In [1]:
import numpy as np
import pandas as pd

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [5]:
df = pd.read_csv('covid_toy.csv')

In [6]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [9]:
df['cough'].value_counts()

cough
Mild      62
Strong    38
Name: count, dtype: int64

In [10]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

In [13]:
X_train

,age,gender,fever,cough,city
75,5,Male,102.0,Mild,Kolkata
36,38,Female,101.0,Mild,Bangalore
91,38,Male,NaN,Mild,Delhi
99,10,Female,98.0,Strong,Kolkata
64,42,Male,104.0,Mild,Mumbai
...,...,...,...,...,...
54,60,Female,99.0,Mild,Mumbai
19,42,Female,NaN,Strong,Bangalore
48,66,Male,99.0,Strong,Bangalore
2,42,Male,101.0,Mild,Delhi


In [19]:
si = SimpleImputer()
X_train_fever=si.fit_transform(X_train[['fever']])
X_test_fever=si.fit_transform(X_test[['fever']])

In [23]:
ordi = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough=ordi.fit_transform(X_train[['cough']])
X_test_cough=ordi.fit_transform(X_test[['cough']])
X_train_fever.shape

(80, 1)

In [25]:
ohe = OneHotEncoder(drop='first',sparse=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])
X_train_gender_city.shape

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 4)

In [26]:
X_train_age = X_train[['age']].values

In [27]:
X_test_age = X_test[['age']].values

In [28]:
X_train_age.shape

(80, 1)

In [29]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_gender_city),axis=1)

In [31]:
X_train_transformed.shape

(80, 7)

In [32]:
from sklearn.compose import ColumnTransformer


In [33]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [35]:
transformer.fit_transform(X_train).shape

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


(80, 7)

In [37]:
transformer.transform(X_test).shape

(20, 7)